# Data Wrangling Project: API Mini-Project

This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?limit=1&api_key='
r = requests.get(url + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data = r.json()

In [5]:
data.keys()

dict_keys(['dataset'])

In [6]:
data['dataset'].keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### Task 1:Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

In [7]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key='
r = requests.get(url + API_KEY)

### Task 2: Convert the returned JSON object into a Python dictionary

In [8]:
afx_x_2017 = r.json()

In [9]:
type(afx_x_2017)

dict

### Task 3: Calculate what the highest and lowest opening prices were for the stock in this period

In [10]:
# Extract subset using key 'data' from the nested dictionary 
afx_x_2017_data = afx_x_2017['dataset']['data']

In [11]:
# Approach 1 using dictionary comprehension to create a dictionary with date being keys and open prices being values
open_dict = {row[0]: row[1] for row in afx_x_2017_data if row[1] is not None}

In [12]:
print("The highest opening prices was ", max(open_dict.values()), "in 2017")
print("The lowest opening prices was ", min(open_dict.values()), "in 2017")

The highest opening prices was  53.11 in 2017
The lowest opening prices was  34.0 in 2017


In [13]:
open_list = [row[1] for row in afx_x_2017_data if row[1] is not None]

In [14]:
max(open_list)

53.11

In [15]:
min(open_list)

34.0

### Task 4: What was the largest change in any one day (based on High and Low price)?

In [16]:
# Using list comprehension to create a list of daily price change using high price minus low price
daily_change = [row[2] - row[3] for row in afx_x_2017_data]

In [17]:
print("The largest change in any one day (based on High and Low price) in 2017 was ", round(max(daily_change), 2))

The largest change in any one day (based on High and Low price) in 2017 was  2.81


### Task 5: What was the largest change between any two days (based on Closing Price)?

In [18]:
# Create a list with price changes between any two days in 2017
between_day_change = []
for i in range(len(afx_x_2017_data) - 1):
    change = afx_x_2017_data[i+1][4] - afx_x_2017_data[i][4]
    between_day_change.append(change)

In [19]:
print('The largest change between any two days (based on Closing Price) in 2017 was {}'.format(
    (round(max(between_day_change), 2))))

The largest change between any two days (based on Closing Price) in 2017 was 2.56


### Task 6: What was the average daily trading volume during this year?

In [20]:
# Extract daily trading volumn using list comprehension
trading_volume = [row[6] for row in afx_x_2017_data]

In [21]:
avg_volume = sum(trading_volume)/len(trading_volume)
print("The average daily trading volume in 2017 was {}".format(round(avg_volume, 2)))

The average daily trading volume in 2017 was 89124.34


### Task 7: (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [22]:
# Define my own function for calculating median
def median(l):
    l.sort()
    if len(l) % 2 == 0:
        median = (l[len(l)/2 - 1] + l[len(l)/2])/2
    else:
        median = l[len(l)//2]
    return median

In [23]:
print("The median trading volume during 2017 was {}".format(median(trading_volume)))

The median trading volume during 2017 was 76286.0
